# CFPKM

In [23]:
import os
import random
from hashlib import sha256

# Stałe
SEEDSIZE = 48
ERROR_LENGTH = 4
SECRETVAL_LENGTH = 4
COFSIZE = 1024
M = 4
N = 4
RANGE = 256
Q = 12289

# Struktura reprezentująca wielomian
class Pol:
    def __init__(self):
        self.QD = [0] * (N * N)
        self.L = [0] * N
        self.C = 0

# Implementacja funkcji generującej losowe bajty
def randombytes(length):
    return os.urandom(length)

# Implementacja funkcji generującej system wielomianów
def polgen(f, m, n):
    for i in range(m):
        cofval = [randombytes(4) for _ in range((N * (N + 1) // 2) + N + 1)]
        count = 0
        for j in range(n):
            for k in range(n):
                if k > j:
                    f[i].QD[k * n + j] = 0
                else:
                    f[i].QD[k * n + j] = int.from_bytes(cofval[count], 'big') % COFSIZE
                    count += 1

        for j in range(n):
            f[i].L[j] = int.from_bytes(cofval[count], 'big') % COFSIZE
            count += 1

        f[i].C = int.from_bytes(cofval[count], 'big') % COFSIZE

# Implementacja funkcji pakującej klucz prywatny
def pack_sk(sk, sa, seed):
    sk[:SEEDSIZE] = seed
    sk[SEEDSIZE:] = sa

# Implementacja funkcji pakującej klucz publiczny
def pack_pk(pk, b1, seed):
    pk[:SEEDSIZE] = seed
    for i in range(M):
        for j in range(8):
            pk[SEEDSIZE + i * 8 + j] = (b1[i] >> (j * 8)) & 0xFF

# Implementacja funkcji pakującej tekst zaszyfrowany
def pack_ct(ct, b2, c):
    ct[:M] = c
    for i in range(M):
        for j in range(8):
            ct[M + i * 8 + j] = (b2[i] >> (j * 8)) & 0xFF

# Implementacja funkcji odpakowującej klucz prywatny
def unpack_sk(sa, seed, sk):
    seed[:] = sk[:SEEDSIZE]
    sa[:] = sk[SEEDSIZE:]

# Implementacja funkcji odpakowującej klucz publiczny
def unpack_pk(b1, seed, pk):
    seed[:] = pk[:SEEDSIZE]
    b1[:] = [int.from_bytes(pk[SEEDSIZE + i * 8 : SEEDSIZE + (i + 1) * 8], 'big') for i in range(M)]

# Implementacja funkcji odpakowującej tekst zaszyfrowany
def unpack_ct(b2, c, ct):
    c[:] = ct[:M]
    b2[:] = [int.from_bytes(ct[M + i * 8 : M + (i + 1) * 8], 'big') for i in range(M)]

# Implementacja funkcji oceny wielomianu
def evaluate_poly(unPoly, pValue):
    result1 = 0
    result2 = 0
    tabResult1 = [0] * N

    for j in range(N):
        for i in range(N):
            tabResult1[j] += pValue[i] * unPoly.QD[i * N + j]
        result1 += tabResult1[j] * pValue[j]

    for i in range(N):
        result2 += unPoly.L[i] * pValue[i]

    result1 += result2 + unPoly.C

    return result1

# Implementacja funkcji oceny systemu wielomianów
def eval_sys(pSyst, pValue):
    result = [0] * M
    for i in range(M):
        result[i] = evaluate_poly(pSyst[i], pValue)
    return result

# Implementacja funkcji zaokrąglającej
def rounding(in_val):
    rem = (in_val + (2 ** (256 - 1))) % COFSIZE
    return rem >> 256

# Implementacja funkcji CrossRound
def kem_crossround1(in_val):
    rem = in_val >> (256 - 1)
    return rem % 2

# Implementacja funkcji pakowania tekstów
def pack_str(S):
    return S.encode('utf-8')

# Implementacja funkcji odpakowywania tekstów
def unpack_str(S):
    return S.decode('utf-8')

# Implementacja funkcji generującej klucze
def generate_keypair():
    seed = randombytes(SEEDSIZE)
    f1 = [Pol() for _ in range(M)]
    polgen(f1, M, N)
    sa = randombytes(N * SECRETVAL_LENGTH)
    sa = [sa[i] % RANGE for i in range(N)]
    e1 = randombytes(M * ERROR_LENGTH)
    e1 = [e1[i] % RANGE for i in range(M * ERROR_LENGTH)]
    b1 = eval_sys(f1, sa)
    b1 = [(b1[i] + e1[i]) % COFSIZE for i in range(M)]
    pk = bytearray(CRYPTO_PUBLICKEYBYTES)
    sk = bytearray(CRYPTO_SECRETKEYBYTES)
    pack_sk(sk, sa, seed)
    pack_pk(pk, b1, seed)
    return pk, sk


# Implementacja funkcji szyfrowania
def encrypt(pk, plaintext):
    seed = os.urandom(SEEDSIZE)
    randombytes_init(seed, None, 256)

    # Generowanie losowej wartości 'sa' dla tekstu zaszyfrowanego
    sa = [random.randint(0, RANGE - 1) for _ in range(N)]

    # Generowanie wektorów błędów e1 i e2
    e1 = [random.randint(0, RANGE - 1) for _ in range(M)]
    e2 = [random.randint(0, RANGE - 1) for _ in range(M)]

    # Ocena wielomianów 'f1' i 'f2' dla 'sa'
    b1 = [evaluate_poly(poly, sa) for poly in pk]
    b2 = [evaluate_poly(poly, sa) for poly in pk]

    # Dodanie wektorów błędów do 'b1' i 'b2'
    b1 = [(b1[i] + e1[i]) % Q for i in range(M)]
    b2 = [(b2[i] + e2[i]) % Q for i in range(M)]

    # Zaokrąglenie wartości w 'b2'
    c = [kem_crossround1(val) for val in b2]

    ct = bytearray(CRYPTO_CIPHERTEXTBYTES)
    pack_ct(ct, b2, c)
    ss = [rounding(val) for val in b2]

    return ct, ss

# Implementacja funkcji deszyfrowania
def decrypt(sk, ct):
    seed = sk[:SEEDSIZE]
    sa = sk[SEEDSIZE:]

    b1 = [0] * M
    c = [0] * M

    unpack_ct(b1, c, ct)

    # Ocena wielomianów 'f2' dla 'sa'
    b2 = [evaluate_poly(poly, sa) for poly in sk]

    # Obliczenie wektora błędów e = c - b2
    e = [(c[i] - b2[i]) % Q for i in range(M)]

    ss = [rounding(val) for val in e]

    return ss